# Riiid! Answer Correctness Prediction Challenge 2020
## 1.Description
<a href="https://ibb.co/w4cjKg9"><img src="https://i.ibb.co/gJ4hD95/image.png" alt="image"  width="4777" height="1190"></a>

Riiid is an AI solutions provider in the field of education market and empowers global education players to rethink traditional ways of learning leveraging AI. With a strong belief in equal opportunity in education, Riiid launched an AI tutor based on deep-learning algorithms in 2017 that attracted more than one million South Korean students. This year, the company released EdNet, the world’s largest open database for AI education containing more than 100 million student interactions.


In this challenge we have to predict whether students are able to answer their next questions correctly. You'll be provided with the same sorts of information a complete education app would have: that student's historic performance, the performance of other students on the same question, metadata about the question itself, and more.



## 2.Problem Statement
* To predict whether students are able to answer their next questions correctly.
* It is a binary classification problem,for a given historic performance data of a student predict whether he/she will able to answer the next question correctly or not.

## 2.Bussiness OBjectives/Constraints

## 3.Data Description


<b>1.train.csv</b>

<pre><b>row_id:</b>(int64) ID code for the row.

timestamp: (int64) the time in milliseconds between this user interaction and the first event completion from that user.

user_id: (int32) ID code for the user.

content_id: (int16) ID code for the user interaction

content_type_id: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

task_container_id: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

user_answer: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

answered_correctly: (int8) if the user responded correctly. Read -1 as null, for lectures.

prior_question_elapsed_time: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

prior_question_had_explanation: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.</pre>

<b>2.questions.csv:</b> (metadata for the questions posed to users)

<pre> question_id: foreign key for the train/test content_id column, when the content type is question (0).

bundle_id: code for which questions are served together.

correct_answer: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

part: the relevant section of the TOEIC test.

tags: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.</pre>

<b>3.lectures.csv:</b> (metadata for the lectures watched by users as they progress in their education.)

<pre>lecture_id: foreign key for the train/test content_id column, when the content type is lecture (1).

part: top level category code for the lecture.

tag: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

type_of: brief description of the core purpose of the lecture</pre>

example_test_rows.csv Three sample groups of the test set data as it will be delivered by the time-series API. The format is largely the same as train.csv. There are two different columns that mirror what information the AI tutor actually has available at any given time, but with the user interactions grouped together for the sake of API performance rather than strictly showing information for a single user at a time. Some users will appear in the hidden test set that have NOT been presented in the train set, emulating the challenge of quickly adapting to modeling new arrivals to a website.

prior_group_responses (string) provides all of the user_answer entries for previous group in a string representation of a list in the first row of the group. All other rows in each group are null. If you are using Python, you will likely want to call eval on the non-null rows. Some rows may be null, or empty lists.

prior_group_answers_correct (string) provides all the answered_correctly field for previous group, with the same format and caveats as prior_group_responses. Some rows may be null, or empty lists.

## 4.Performance Metric
* Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target i.e AUC Score.

## 5.Exploratory Data Analysis

In [ ]:
# Importing Libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab 
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<a id="1"></a>
<h2 style='background:black; border:0; color:white'><center>1. train.csv<center><h2>

In [ ]:
%%time

dtypes = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean"
}

data = pd.read_csv("../input/riiid-test-answer-prediction/train.csv", dtype=dtypes)

print("Train size:", data.shape)

In [ ]:
data.head(50)

In [ ]:
data.tail()

In [ ]:
#Brief Summary of the given Dataset
data.info(verbose=True)

* Total no. of rows = 101230332
* Total no. of columns = 10 

In [ ]:
#checking memory usage
data.memory_usage(deep=True)

* row_id is taking lot of memory,also it is not an important feature.

In [ ]:
#droping row_id from the data
data.drop('row_id',axis =1,inplace =True)

In [ ]:
data.describe().style.background_gradient(cmap='Greys')

In [ ]:
#Checking Missing Values in the data
data.isnull().sum()

In [ ]:
#creating figure,axex,title&labels
x = list(data.isnull().sum())
y= list(data.columns)
X = np.arange(len(y))
Y=[0.25,1.25,2.25,3.25,4.25,5.25,6.25,7.25,8.25]
fig = plt.figure(figsize=(24,5))
plt.ylabel('Counts')
plt.title('No. of Null Values Vs total ')
plt.xticks(Y,y)
#creating bar plots
plt.bar(X+0.0,x ,color = 'black', width = 0.25,alpha=0.5,label='Null Values')
plt.bar(X+0.25,[101230332,101230332,101230332,101230332,101230332,101230332,101230332,101230332,101230332], color = 'orange', width = 0.25,alpha=0.6,label='Total')
plt.legend()
plt.show()

In [ ]:
#Correlation matrix 
corr_matrix = data.corr()
corr_matrix["answered_correctly"].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(18,8))
sns.set(font_scale=1.3)
cmap = sns.light_palette("#2f3b39",as_cmap=True)
sns.heatmap(corr_matrix, cmap=cmap,annot=True)
plt.title("  Correlation Matrix of the features",fontsize=19)
plt.show()


In [ ]:
dup_rows = data[data.duplicated(['timestamp','user_id','content_id'])]
dup_rows.head(10)

In [ ]:
data.loc[(data['user_id'] == 115) & (data['timestamp'] == 0)]

In [ ]:
data.loc[(data['user_id'] == 338684437) & (data['timestamp'] == 0)]

In [ ]:
data.loc[(data['user_id'] == 1615528747) & (data['timestamp'] == 0)]

In [ ]:
data.loc[(data['user_id'] == 18971157) & (data['timestamp'] == 0)]

In [ ]:
#Deduplication of entries
final= data.drop_duplicates(keep='first', inplace=False)
final.shape

In [ ]:
data.loc[users.duplicated(keep='first'), :]

In [ ]:

chunker = pd.read_csv(AUTHORS_PATH, names=['Author ID', 'Author name'], encoding='utf-8', chunksize=10000000)

In [ ]:
%%time

dtypes = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean"
}

chunker = pd.read_csv("../input/riiid-test-answer-prediction/train.csv", dtype=dtypes,chunksize = 10000000)

ids = set()
for chunk in chunker:
    chunk.drop_duplicates(keep='first', inplace=False)
    
    chunk = chunk[~chunk['row_id'].isin(ids)]    
    ids.update(chunk['row_id'].values)    

Observations:

## Univariate Analysis

### **timestamp**

In [ ]:
%matplotlib inline
fig,ax = plt.subplots(figsize=(16,6))
ax.ticklabel_format(style ='Plain')
plt.hist(data['timestamp'], bins= 50,edgecolor='black',color='grey');
plt.xlabel('timestamp',fontsize=18)
plt.ylabel('count',fontsize=18)
ax.tick_params(labelsize=14)
plt.title("Count of 'timestamp' ",fontsize=20)
ax.ticklabel_format(useOffset=False)
plt.grid()
plt.ioff()

In [ ]:
%matplotlib inline
sns.set_style('ticks')
fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sns.distplot(data['timestamp'], color='#2f3b39')
plt.xlabel('PDF of timestamp')
sns.despine()
plt.show()

In [ ]:
from datetime import datetime
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms',origin='2017-1-1')
data['month']=(data.timestamp.dt.month)
data['day']=(data.timestamp.dt.day)
data=data.drop(columns=['timestamp'])

In [ ]:
#calculating total fare amount values at each percntile 90,91,92,93,94,95,96,97,98,99,100
for i in range(90,100):
    var = data["timestamp"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print("100 percentile value is ",var[-1])

### **user_id**

In [ ]:
print("Total number of unique user_id:",len((data.user_id).unique()))

In [ ]:
#creating figure,axex,title&labels
fig = plt.figure()


plt.ylabel('No of User_Ids')
plt.title('unique user_ids Vs total user_ids')
plt.xticks([10,10.5],['unique user_id','Total user_id'])
#creating bar plots
plt.bar(10,393656, color = 'black', width = 0.25,alpha=0.5,label='unique user_id')
plt.bar(10.25,101230332, color = 'orange', width = 0.25,alpha=0.6,label='Total user_id')
plt.legend()
plt.show()

In [ ]:
temp = pd.DataFrame(data.groupby('user_id')['answered_correctly'].agg(lambda x: x.eq(1).sum())).reset_index()


In [ ]:
temp

In [ ]:
top_user = temp.sort_values(['answered_correctly'], ascending=False).head(25)
top_user.head()

In [ ]:
plt.figure(figsize=(15, 15))
sns.barplot(x='user_id', y= 'answered_correctly', data= top_user,palette="Blues")
plt.title(f'Top {25} the most active users', fontsize=14)

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data['content_type_id'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Percentage content_type_id Distribution')
ax[0].set_ylabel('Count')
ax[1].ticklabel_format(style ='Plain')
sns.countplot('content_type_id',data=data,ax=ax[1],order=data['content_type_id'].value_counts().index)
ax[1].set_title('Count of content_type_id')
plt.show()

In [ ]:
data.task_container_id.value_counts()

In [ ]:
data.user_answer.value_counts()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
train['user_answer'].value_counts().plot.pie(explode=[0,0.1,0.1,0.1,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Percentage user_answer Distribution')
ax[0].set_ylabel('Count')
sns.countplot('user_answer',data=train,ax=ax[1],order=train['user_answer'].value_counts().index)
ax[1].set_title('Count of user_answer')
plt.show()

In [ ]:
N = 25 # number of users

user_freq = data['user_id'].value_counts().reset_index()
user_freq.columns = [
    'user_id', 
    'count'
]

# Add ' - ' to convert user_id to str and not sort
user_freq['user_id'] = user_freq['user_id'].astype(str) + ' - '
user_freq = user_freq.sort_values(['count'], ascending=False).head(N)

plt.figure(figsize=(15, 15))
sns.barplot(x='count', y='user_id', data=user_freq,palette="Greys")
plt.title(f'Top {N} the most active users', fontsize=14)

In [ ]:
ds = train['user_id'].value_counts().reset_index()
ds.columns = ['user_id', 'count']
ds['user_id'] = ds['user_id'].astype(str) + '-'
ds = ds.sort_values(['count'])
top_40 = ds.tail(40)


fig,ax = plt.subplots(figsize=(12,8))
sns.barplot(top_40['count'],top_40['user_id'])

In [ ]:
%matplotlib inline
sns.set_style('ticks')
fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sns.distplot(list(data['user_id']), color='#2f3b39')
plt.xlabel('PDF of user_id')
sns.despine()
plt.show()

In [ ]:

%matplotlib inline
sns.set_style('ticks')
fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sns.distplot(list(data['content_id']), color='#16A085')
plt.xlabel('PDF of conten_id')
sns.despine()
plt.show()

In [ ]:
plt.boxplot(data['content_id'])
plt.ylabel('content_id')
plt.show()

In [ ]:
temp = pd.DataFrame(data.groupby('user_id')['answered_correctly'].agg(lambda x: x.eq(1).sum())).reset_index()

In [ ]:
temp.sort_

In [ ]:
data['content_type_id'].value_counts().plot(kind='bar',color=['black','red'],alpha=0.5)

In [ ]:
data['answered_correctly'].value_counts().plot(kind='bar',color=['black','red','yellow'],alpha=0.5)

<a id="1"></a>
<h2 style='background:black; border:0; color:white'><center>2. lectures.csv<center><h2>

In [ ]:
lectures = pd.read_csv("../input/riiid-test-answer-prediction/lectures.csv")
lectures.head(10)

In [ ]:
lectures.info()

<a id="1"></a>
<h2 style='background:black; border:0; color:white'><center>3. questions.csv<center><h2>

In [ ]:
questions = pd.read_csv("../input/riiid-test-answer-prediction/questions.csv")
questions.head(10)

In [ ]:
questions.info()

In [ ]:
questions.loc[questions['tags'] == '134']

In [ ]:
print("Total number of unique user_id:",((questions.part).unique()))

In [ ]:
print("Total number of unique user_id:",((lectures.tag).unique()))

In [ ]:
import math
data_log = [math.log(i) for i in list(data['timestamp']) if i>0]

In [ ]:
sns.kdeplot(data= data_log)